## BAND Framework Toy Model: Ball Drop

### Part 1: Emulation

This is Alan's version of [Surmise's Example 1](https://nbviewer.jupyter.org/github/surmising/surmise/blob/master/examples/Example1/Example1_nb.ipynb). In this example, we will emulate the results from two different **Ball Drop models**: turning on and off the drag force (quadratic). 

First, we import the required libraries.

In [31]:
import numpy as np 
import scipy.stats as sps     #Do I need to have scipy installed? Yes.
import matplotlib.pyplot as plt

from emulation import emulator # Import an emulator which I should write/understand

### Models

The height of a large falling ball from a tower is recorded at different times until it reaches the ground. The same phenomenon is described by two different models.


#### Model 1 ($M_{1}$)

We describe the falling ball without drag force. Here, the gravitational force causes the object to have acceleration g downward. The height is given by *$h = h_{0} - \frac{1}{2}gt^{2}$* where $h_{0}$ is the initial height and $t$ is the time to reach $h$. The inputs used are:

- time $t$,
- initial height $h_{0}$, and
- gravity $g$.

For the next function, we use $x = (t, h_{0})$ and $\theta = g$; $f$ represents the model outputs.

In [32]:
def balldropmodel_grav(x, theta):
    f = np.zeros((theta.shape[0],x.shape[0]))
    for k in range(0, theta.shape[0]):
        t = x[:, 0]
        h0 = x[:, 1]
        g = theta[k]
        f[k, :] = h0 - (g / 2) * (t ** 2)
    return f.T

#Playing around, checking outputs, and how to arrange inputs:

#x = np.array([[1,10],[1,40], [1,8]])
#theta = np.array([9.8, 10])
#h = balldropmodel_grav(x, theta)
#print(h)

### Model 2 ($M_{2}$)

$M_{2}$ here is different from that in the [surmise](https://nbviewer.jupyter.org/github/surmising/surmise/blob/master/examples/Example1/Example1_nb.ipynb) example. Surmise uses a drag force that is linear with the drag coefficient. Here, we'll use the quadratic drag force of ball drop in accordance with the [BAND Manifesto](https://arxiv.org/abs/2012.07704).

When $t$ is large enough, we can approximate $h$ as a straight line: $h = h_{0} + v_{ter}t + ¿c?$. We will use the following inputs:

- time $t$,
- drag coefficient $\gamma$,
- gravity $g$,
- ball's diameter $D$,
- ball's mass $m$, and 
- initial height $h_{0}$.

Here $x = (t, h_{0})$ and $\theta = (g, \gamma, m, D)$; $f$ represents the model outputs. 

${\bf Questions:}$ what is the constant c?

In [33]:
def balldropmodel_quad(x, theta):
    f = np.zeros((theta.shape[0], x.shape[0]))
    for k in range(0, theta.shape[0]):
        t = x[:, 0]
        h0 = x[:, 1]
        g = theta[k, 0]
        gamma = theta[:, 1]
        m = theta[:, 2]
        D = theta[:, 3]
        c = gamma[k]*(D[k]**2)
        vter = np.sqrt(m[k] * (g/c))
        
        if gamma[k] > 0.0 :
            f[k, :] = h0 - ((vter**2)/g)*np.log(np.cosh(g*t/vter))
        else :
            f[k, :] = balldropmodel_grav(x, theta)
    return f.T

#Playing around, checking outputs, and how to arrange inputs:

#x = np.array([[1,10],[1,40]])
#theta = np.array([[9.8, 0.002, 1, 1], [9.8, 0.001, 1, 1]])
#h = balldropmodel_quad(x, theta)
#print(h)

#It works and agrees with M1 in the limit gamma -> 0.

Let me now implement the previous models in the same input space as the Surmise code uses. i.e. on an input grid $t \in [0.1,4.2]$ and $h_{0} \in \{25,50\}$.

In [34]:
# the time vector of interest
tvec = np.concatenate((np.arange(0.1, 4.3, 0.1), np.arange(0.1, 4.3, 0.1)))
#print(tvec)

# the drop heights vector of interest
h0vec = np.concatenate((25 * np.ones(42), 50 * np.ones(42)))  

# the input of interest
xtot = (np.vstack((tvec, h0vec)).T).astype('object')
#print(xtot)

xtotv = xtot.astype('float')
xtot[xtot[:,1] == 25, 1] = 'lowdrop'
xtot[xtot[:,1] == 50, 1] = 'highdrop'
#print(xtot)

print(np.shape(xtot))



(84, 2)



### Prior specification

Typically, the researchers have a prior knowledge about the unknown parameters in the form of a prior distribution.

In $M_{1}$, we assume 

- $g$ is gamma-distributed random variable with shape $\alpha$, location $\mu$ and rate $\beta$ parameters such that $g$ $\sim$ $\Gamma (\alpha,\mu,\beta)$ with $\alpha$ = 2, $\mu$ = 0, $\beta$ = 5.

In $M_{2}$, we assume 

-  $g$ has a gamma distribution with shape $\alpha$, location $\mu$ and rate $\beta$ parameters such that $v_{ter}$ $\sim$ $\Gamma (\alpha,\mu,\beta)$ with \$\alpha$ = 2, $\mu$ = 0, $\beta$ = 5,

-  $\gamma$ has a gamma distribution with shape $\alpha$, location $\mu$ and rate $\beta$ parameters such that $\gamma$ $\sim$ $\Gamma (\alpha,\mu,\beta)$ with $\alpha$ = 2, $\mu$ = 0, $\beta$ = 5.

In [35]:
class priorphys_grav:
    """ This defines the class instance of priors provided to the method. """
    def lpdf(theta):
        return (sps.gamma.logpdf(theta[:, 0], 2, 0, 5)).reshape((len(theta), 1))
    
    def rnd(n):
        return np.reshape(sps.gamma.rvs(2, 0, 5, size=n), (-1, 1))  # gravity

In [36]:
class priorphys_quad:
    """ This defines the class instance of priors provided to the method. """
    def lpdf(theta):
        return (sps.gamma.logpdf(theta[:, 0], 2, 0, 5), #gravity
                sps.norm.logpdf(theta[:, 1], 0, 5)).reshape((len(theta), 1))   # gamma


    def rnd(n, m, D):
        return np.vstack((sps.gamma.rvs(2, 0, 5, size=n),  # gravity
                          sps.gamma.rvs(2, 0, 0.5, size=n), m * np.ones(50), D * np.ones(50))).T  # gamma

In [37]:
#NEED TO PLOT THIS. Exploring PDFs notebook course on Bayesian Stats.

x = np.linspace(0, 20, 201)
#y = sps.gamma.rvs(2, 0 , 0.2, 10)
#print(y)
y = sps.gamma.rvs(2, 0, 5, 10)
print(y)

[ 5.99920432 12.39869868  1.41607809  2.96092491 14.61478041  4.15317234
  9.44995381  9.47534573  4.81257888 12.59846824]


Now, we draw 50 random parameters.

In [38]:
# draw 50 random parameters from the prior
theta_grav = priorphys_grav.rnd(50) 
print(np.shape(theta_grav))
#print(theta_grav)

# draw 50 random parameters from the prior
m = 1
D = 1
theta_quad = priorphys_quad.rnd(50, m, D) #Last two entries are m and D all ones!
print(np.shape(theta_quad))

#printing things to check
print(theta_quad)

(50, 1)
(50, 4)
[[ 3.69565294  1.68666508  1.          1.        ]
 [ 2.77856859  0.70398642  1.          1.        ]
 [ 7.57576831  0.11273683  1.          1.        ]
 [11.80590055  0.32705321  1.          1.        ]
 [ 6.8536839   1.17311726  1.          1.        ]
 [ 7.5008578   0.97480817  1.          1.        ]
 [ 8.52739049  0.56660419  1.          1.        ]
 [ 3.28344383  0.70996709  1.          1.        ]
 [ 6.88260919  0.36084772  1.          1.        ]
 [13.6153883   0.6505749   1.          1.        ]
 [22.25056205  0.49825289  1.          1.        ]
 [11.87558963  0.24139942  1.          1.        ]
 [11.70112349  1.43609312  1.          1.        ]
 [ 6.64585789  0.557312    1.          1.        ]
 [ 4.28620251  1.24794117  1.          1.        ]
 [ 5.00236001  0.38216324  1.          1.        ]
 [25.80817488  0.15774928  1.          1.        ]
 [ 1.51688765  0.39535757  1.          1.        ]
 [ 1.66764484  1.29991718  1.          1.        ]
 [ 7.83779451  

Let's evaluate the computer models $M_{1}$ and $M_{2}$ at those random points generated above, and obtain $m \times n$ computer model output matrix ${\bf f}$.

In [39]:
# create a computer experiment to build an emulator for the gravity simulation
f_grav = balldropmodel_grav(xtotv, theta_grav)  
print(np.shape(f_grav))

# create a computer experiment to build an emulator for the linear simulation
f_quad = balldropmodel_quad(xtotv, theta_quad) 
print(np.shape(f_quad))

#printing things to check
#print("\n First: \n \n", f_grav)
#print("\n Second: \n \n ", f_quad)
#print(f_quad.ndim)
#print(theta_quad.ndim)
#print(xtot.ndim)

(84, 50)
(84, 50)


## Model emulation

Now, we will emulate the models $M_{1}$ and $M_{2}$. We will use PCGP.

##### Comment to discuss:

I had to fix theta_quad ad hoc here to take only the 2 parameters but shouldn't the code be able to handle as many given parameters?

In [40]:
# build an emulator for the quadratic simulation
emu_quad_1 = emulator(x=xtot, theta=theta_quad, f=f_quad, method='PCGP') 


# build an emulator for the gravity simulation
emu_grav_1 = emulator(x=xtot, theta=theta_grav, f=f_grav, method='PCGP')

##printing things to check
#print(emu_quad_1)

## Comparison of emulation and models 

Let's first generate random draws of parameters, and evaluate the computer model at those values.

In [19]:
# (Test) draw 50 random parameters from the prior
theta_grav_test = priorphys_grav.rnd(50) 

# (Test) draw 50 random parameters from the prior
theta_quad_test = priorphys_quad.rnd(50, m, D)  

# (Test) the value of the gravity simulation
f_grav_test = balldropmodel_grav(xtotv, theta_grav_test)  

# (Test) the value of the linear simulation
f_quad_test = balldropmodel_quad(xtotv, theta_quad_test) 

## Printing things to check
#print("\n First: \n \n", theta_grav_test)
#print("\n Second: \n \n", theta_quad_test)
#print(theta_quad_test[:, 0:2])

Then, let's get the predict object of the emulator for the hold-out data:

In [24]:
pred_grav_1 = emu_grav_1.predict(x=xtot, theta=theta_grav_test)
pred_quad_1 = emu_quad_1.predict(x=xtot, theta=theta_quad_test)

## printing things to check
#print(pred_quad_1)

A emulation prediction object predict where the code in located in the file  emulation.  The main method are predict.covx, predict.covxhalf, predict.covxhalf_gradtheta, predict.lpdf, predict.mean, predict.mean_gradtheta, predict.rnd, predict.var.  Default of predict() is predict.mean() and predict(s) will run pred.rnd(s). Run help(predict) for the document string.


In [25]:
# get the prediction means and variances
pred_quad_1_m, pred_quad_1_var = pred_quad_1.mean(), pred_quad_1.var()
pred_grav_1_m, pred_grav_1_var = pred_grav_1.mean(), pred_grav_1.var()

##printing things to check
#print("\n First: \n \n", pred_grav_1_m)
print("\n Second: \n \n", pred_quad_1_m)


 Second: 
 
 [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


Finally, let's observe the sum of squared deviations between the prediction means and the simulated output:

In [23]:
print('Rsq PCGP = ', np.round(1 - np.sum(np.square(pred_quad_1_m - f_quad_test))/np.sum(np.square(f_quad_test.T - np.mean(f_quad_test, axis = 1))), 2))
print('SSE PCGP = ', np.round(np.sum(np.square(pred_quad_1_m - f_quad_test)), 2))

print('Rsq PCGP = ', np.round(1 - np.sum(np.square(pred_grav_1_m - f_grav_test))/np.sum(np.square(f_grav_test.T - np.mean(f_grav_test, axis = 1))), 2))
print('SSE PCGP = ', np.round(np.sum(np.square(pred_grav_1_m - f_grav_test)), 2))

Rsq PCGP =  nan
SSE PCGP =  nan
Rsq PCGP =  -0.16
SSE PCGP =  3188778.43
